In [1]:

from py2neo import Graph,Node,Relationship,NodeMatcher,RelationshipMatcher


In [40]:

class Neo4jDatabase:
    def __init__(self):

        self.graph = Graph('http://10.10.64.190:8919', auth=("neo4j", "123456"))
        self.n_matcher = NodeMatcher(self.graph)
        self.r_matcher = RelationshipMatcher(self.graph)

    def return_all_dataset(self):
        match_str = "match (n) return n"
        res = self.graph.run(match_str)
        return res.data()
    def create_node(self,ent):
        """
        构建一个生物医学实体
        label:entity_type
        id: 实体的唯一ID
        name: 这是Node属性
        """
        ent_name = ent['entity_name']
        ent_id = ent['id']
        entity_type = ent['entity_type']
        ent_norm_id = ent['norm_id']
        ent_norm_mention = ent['norm_name']
        url = ent['url']
        node = Node(ent_name,ent_id=ent_id,label=entity_type,kg_name='gastric_cancer_kg',name=ent_name,url=url,norm_id=ent_norm_id,normalize_ent_name=ent_norm_mention)
        self.graph.create(node)

    def create_relation(self,node1,node2):

        rel = Relationship(node1,'interaction',node2)
        self.graph.create(rel)

    def filter_node_by_entity_type(self,entity_type):
        """
        返回实体类别对应的所有节点
        """
        node = self.n_matcher.match(label=entity_type)
        return node.all()
    def filter_node_all_relationse(self,ent,r_type=None):
        """
        ：param ent:以这个ent为起点来查找他所有的关系
        这个就是返回某个节点所对应的所有关系三员组(一个完整的结果,<n1,r,n2>)
        """
        node = self.n_matcher.match(label='')
        if r_type is None:
            relations = self.r_matcher.match({node})
            return relations.all()
        else:
            relations = self.r_matcher.match({node},r_type=r_type)
            return relations.all()

In [41]:
neo = Neo4jDatabase()

In [42]:
node1 = neo.n_matcher.match(label="Disease")

In [47]:
type(node1)

py2neo.matching.NodeMatch

In [ ]:
dir(node1)

In [55]:
node1.first()

Node('感冒', label='Disease', name='感冒')

In [56]:
r_res = neo.r_matcher.match({node1.first()})

In [60]:
r_res.all()

[治疗(Node('阿斯匹林', label='Drug', name='阿斯匹林'), Node('感冒', label='Disease', name='感冒'))]

In [92]:
r_res.all()[0].relations

AttributeError: '治疗' object has no attribute 'relations'

In [95]:
node1,node2 = r_res.all()[0].nodes

In [96]:
node1

Node('阿斯匹林', label='Drug', name='阿斯匹林')

In [97]:
node2

Node('感冒', label='Disease', name='感冒')

In [101]:
r_res = neo.r_matcher.match({node1,node2})

In [104]:
dir(r_res.all()[0])

['_Subgraph__nodes',
 '_Subgraph__relationships',
 '_Walkable__sequence',
 '__add__',
 '__and__',
 '__bool__',
 '__class__',
 '__contains__',
 '__db_create__',
 '__db_delete__',
 '__db_exists__',
 '__db_merge__',
 '__db_pull__',
 '__db_push__',
 '__db_separate__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iand__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__isub__',
 '__iter__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__uuid__',
 '__walk__',
 '__weakref__',
 '__xor__',
 '_graph',
 '_is_bound',
 '_stale',
 'clear',
 'copy',
 'end_node',
 'fromkeys',
 'get',
 'graph',
 'identity',
 'items',
 'keys',
 'labels',
 'nodes',
 'pop',
 'pop

In [125]:
list(r_res.all()[0].types())[0]

'治疗'